In [2]:
import form
import re
f = form.open("form -l",keep_log=100)

In [3]:
k = [[" 1 ", " 1 ", " 1 ", " 1 "], 
     [" 1 ", " 1 ", " i_", "-i_"], 
     [" 1 ", "-i_", " 1 ", " i_"], 
     [" 1 ", " i_", "-i_", " 1 "]]


n = [[" 1   ", "cx(i)", "cy(i)", "cz(i)"], 
     ["cx(i)", " 1   ", "cz(i)", "cy(i)"], 
     ["cy(i)", "cz(i)", " 1   ", "cx(i)"], 
     ["cz(i)", "cy(i)", "cx(i)", " 1   "]]

c = ["1","cx","cy","cz"]
s = ["1","sx","sy","sz"]
rules = '\n'.join('id ifmatch->endarg '+c[i]+'(i?)*'+s[j]+'(i?) = '+k[i][j]+'*'+str(n[j][i])+';' 
       for i in range(1,4) for j in range(1,4))
#for i in range(4):
#    print()
#    for j in range(4):
#        print('id m'+str(i)+'*m'+str(j)+' = '\
#              +k[i][j]+'*'+str(n[j][i])+';')
print(rules)

id ifmatch->endarg cx(i?)*sx(i?) =  1 * 1   ;
id ifmatch->endarg cx(i?)*sy(i?) =  i_*cz(i);
id ifmatch->endarg cx(i?)*sz(i?) = -i_*cy(i);
id ifmatch->endarg cy(i?)*sx(i?) = -i_*cz(i);
id ifmatch->endarg cy(i?)*sy(i?) =  1 * 1   ;
id ifmatch->endarg cy(i?)*sz(i?) =  i_*cx(i);
id ifmatch->endarg cz(i?)*sx(i?) =  i_*cy(i);
id ifmatch->endarg cz(i?)*sy(i?) = -i_*cx(i);
id ifmatch->endarg cz(i?)*sz(i?) =  1 * 1   ;


In [8]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
cfun   cx,cy,cz,cc; *cten
set CC:cx,cy,cz;
nfun   sx,sy,sz,sc;
set SC:sx,sy,sz;

Function S,Comm;
cfun koef;
polyfun koef;

auto Symbols a,b,t;
auto Symbol n,i,j,k; * indices

Local C = Comm(cx(1)*cx(2),cy(1)*cy(3));


*--- overlapping ---
argument Comm;
    multiply S(1,1);
    repeat id S(k?,t?)*cc?CC(i?) = S(max_(k,i),t*cc(i));
endargument;
id Comm(S(k1?$k1max,t1?),S(k2?$k2max,t2?)) = Comm(S(k1,t1),S(k2,t2));
$kmax = max_($k1max,$k2max);
id Comm(S(n1?,a1?),S(n2?,a2?)) = Comm(a1,a2) + 
    sum_(i,1,n2-1,Comm(S(i,1)*a1,a2)) +
    sum_(i,1,n1-1,Comm(a1,S(i,1)*a2));
argument Comm;
    repeat id S(i?,t?)*cc?CC(j?) = S(i,t*cc(i+j));
    id S(i?,a?) = a;
endargument;


Print;
.sort
''')
print(re.sub(r'\+','\n+',f.read('C')))
print(re.sub(r'\+','\n+',f.read('$kmax')))
#f.read('C')

Comm(cx(1)*cx(2),cy(1)*cy(3))*koef(1)
+Comm(cx(1)*cx(2),cy(2)*cy(4))*koef(1)
+Comm(cx(2)*cx(3),cy(1)*cy(3))*koef(1)
+Comm(cx(3)*cx(4),cy(1)*cy(3))*koef(1)
3


In [10]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
cfun   cx,cy,cz,cc; *cten
set CC:cx,cy,cz;
nfun   sx,sy,sz,sc;
set SC:sx,sy,sz;

Function S,Comm;
cfun koef;
polyfun koef;

auto Symbols a,b,t;
auto Symbol n,i,j,k; * indices

Local C = Comm(cx(1)*cx(2),cy(1)*cy(3));

*--- overlapping ---
argument Comm;
    multiply S(1,1);
    repeat id S(k?,t?)*cc?CC(i?) = S(max_(k,i),t*cc(i));
endargument;

id Comm(S(k1?$k1max,t1?),S(k2?$k2max,t2?)) = Comm(S(k1,t1),S(k2,t2));
$kmax = 2*$k1max + 2*$k2max;

id Comm(S(n1?,a1?),S(n2?,a2?)) = Comm(a1,a2) + 
    sum_(i,1,n2-1,Comm(S(i,1)*a1,a2)) +
    sum_(i,1,n1-1,Comm(a1,S(i,1)*a2));
argument Comm;
    repeat id S(i?,t?)*cc?CC(j?) = S(i,t*cc(i+j));
    id S(i?,a?) = a;
endargument;

*--- multiply ---
id Comm(a?,b?) = S(a)*b-S(b)*a;
label repeat;
    id ifnomatch->endrepeat S(a?)*cc?CC[n](i?) = S(a*SC[n](i));
    argument;
*        id cc?CC(i?)*sc?SC(i?) = 1;
'''+rules+
'''
        id sc?SC[n?](i?) = CC[n](i);
        label endarg;
    endargument;
goto repeat;
label endrepeat;
id S(a?) = a;
.sort

*--- trim ---
if(match(cc?CC(i?)));
    multiply S($kmax,1);
    repeat id S(n?,t?)*cc?CC(i?) = S(min_(n,i),t*cc(i));
    id ifmatch->endif S(1,t?) = t;
        id S(n?,t?) = S(n-1,1)*t;
        repeat id S(n?,t?)*cc?CC(i?) = S(n,t*cc(i-n));
        id S(n?,t?) = t;
    label endif;
endif;

Print;
.sort
''')
print(re.sub(r'\+','\n+',f.read('C')))
#f.read('C')

cx(1)*cy(4)*cz(2)*i_*koef(2)
+cx(2)*cy(1)*cz(3)*i_*koef(2)
+cx(2)*cy(3)*cz(1)*i_*koef(2)
+cx(4)*cy(1)*cz(3)*i_*koef(2)


In [4]:
procedures = '''
cfun   cx,cy,cz,cc; *cten
set CC:cx,cy,cz;
nfun   sx,sy,sz,sc;
set SC:sx,sy,sz;
Function S,Comm;
cfun koef;
auto Symbols a,b,t;
auto Symbol n,i,j,k; * indices

*--- overlapping ---
#procedure overlapping
    argument Comm;
        multiply S(1,1);
        repeat id S(k?,t?)*cc?CC(i?) = S(max_(k,i),t*cc(i));
    endargument;
    id Comm(a?,b?) = a*b;
    id S(?ss1)*S(?ss2) = Comm(S(?ss1),S(?ss2));
    id Comm(S(n1?,a1?),S(n2?,a2?)) = Comm(a1,a2) + 
        sum_(i,1,n2-1,Comm(S(i,1)*a1,a2)) +
        sum_(i,1,n1-1,Comm(a1,S(i,1)*a2));
    argument Comm;
        repeat id S(i?,t?)*cc?CC(j?) = S(i,t*cc(i+j));
        id S(i?,a?) = a;
    endargument;
#endprocedure

*id Comm(a?,b?) = S(a)*b-S(b)*a;
*--- multiply ---
#procedure multiply
    label repeat;
        id ifnomatch->endrepeat S(a?)*cc?CC[n](i?) = S(a*SC[n](i));
        argument S;
*        id cc?CC(i?)*sc?SC(i?) = 1;
    '''+rules+'''
            id sc?SC[n?](i?) = CC[n](i);
            label endarg;
        endargument;
    goto repeat;
    label endrepeat;
    id S(a?) = a;
#endprocedure
*.sort

*--- trim ---
#procedure trim
    if(match(cc?CC(i?)));
        multiply S(1000000,1);
        repeat id S(n?,t?)*cc?CC(i?) = S(min_(n,i),t*cc(i));
        id ifmatch->endif S(1,t?) = t;
            id S(n?,t?) = S(n-1,1)*t;
            repeat id S(n?,t?)*cc?CC(i?) = S(n,t*cc(i-n));
            id S(n?,t?) = t;
        label endif;
    endif;
#endprocedure
'''

In [5]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
Sym h,J;

*cfun koef;
*polyfun koef;

S n,n1,n2,x;
Table ident(x?);  Fill ident() = x;
Table prod(n?symbol_,n1?int_,n2?int_,x?);
Fill prod() =
  + theta_(n2-n1) * prod(n,n1,n2-1,x) * ident(x * replace_(n,n2))
  + thetap_(n1-n2)
;

S x,y,z;
Table if(x?int_,y?,z?);
Fill if() = deltap_(x)*y+delta_(x)*z;
'''+procedures+
'''

Table X(n?); Fill X() = cx(1)*prod(i,2,n,cz(i))*cx(n+1);
Table Y(n?); Fill Y() = if(thetap_(n),cy(1)*prod(i,2,n,cz(i))*cy(n+1),-cz(1));
Table M(n?); Fill M() = if(thetap_(n),
    (cx(1)*prod(i,2,n,cz(i))*cy(n+1)+cy(1)*prod(i,2,n,cz(i))*cx(n+1))/2,
    0);
Table Im(n?); Fill Im() = if(thetap_(n),
    (cx(1)*prod(i,2,n,cz(i))*cy(n+1)-cy(1)*prod(i,2,n,cz(i))*cx(n+1))/2,
    0);
Table Ip(n); Fill Ip() = h*X(n) + h*Y(n) - X(n)*replace_(n,n+1) - Y(n)*replace_(n,n-1);
.global
G H = -J*cx(1)*cx(2)-J*h*cz(1);
.store

#define N "50"
L HIp1 = Comm(H,Ip(`N'));
L HIm1 = Comm(H,Im(`N'));
L HX1   = i_*Comm(H,X(`N')) - 4*J*h*M(`N') + 4*J*M(`N'-1);
L HY1   = i_*Comm(H,Y(`N')) + 4*J*h*M(`N') - 4*J*M(`N'+1);
L HM1   = i_*Comm(H,M(`N'))  +2*J*h*X(`N') - 2*J*h*Y(`N') - 2*J*X(`N'+1) + 2*J*Y(`N'-1);


#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim

.sort
''')

print(f.read('HIp1'))
print(f.read('HIm1'))
print(f.read('HX1'))
print(f.read('HY1'))
print(f.read('HM1'))


0
0
0
0
0


In [6]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
'''+procedures+
'''
polyfun koef;

Local C = Comm(cx(1)*cx(2),cy(1)*cy(3));

#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim

Print;
.sort
''')
print(re.sub(r'\+','\n+',f.read('C')))
#f.read('C')

cx(1)*cy(4)*cz(2)*i_*koef(2)
+cx(2)*cy(1)*cz(3)*i_*koef(2)
+cx(2)*cy(3)*cz(1)*i_*koef(2)
+cx(4)*cy(1)*cz(3)*i_*koef(2)


In [24]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
cfun if;
Sym h,J;

*cfun koef;
*polyfun koef;

'''+procedures+
'''
.global

G X(n) = cx(1)*S(sum_(i,2,n,cz(i)))*cx(n+1);
G Y(n) = if(thetap_(n),cy(1)*S(sum_(i,2,n,cz(i)))*cy(n+1),-cz(1));
G M(n) = if(thetap_(n),
    (cx(1)*S(sum_(i,2,n,cz(i)))*cy(n+1)+cy(1)*S(sum_(i,2,n,cz(i)))*cx(n+1))/2,
    0);
G Im(n) = if(thetap_(n),
    (cx(1)*S(sum_(i,2,n,cz(i)))*cy(n+1)-cy(1)*S(sum_(i,2,n,cz(i)))*cx(n+1))/2,
    0);
.store
G Ip(n) = h*X(n) + h*Y(n) - X(n)*replace_(n,n+1) - Y(n)*replace_(n,n-1);
G H = -J*cx(1)*cx(2)-J*h*cz(1);
.store

L HIp1 = Comm(H,Ip(n));
L HIm1 = Comm(H,Im(n));
L HX1   = i_*Comm(H,X(n)) - 4*J*h*M(n) + 4*J*M(n)*replace_(n,n-1);
L HY1   = i_*Comm(H,Y(n)) + 4*J*h*M(n) - 4*J*M(n)*replace_(n,n+1);
L HM1   = i_*Comm(H,M(n))  +2*J*h*X(n) - 2*J*h*Y(n) - 
                            2*J*X(n)*replace_(n,n+1) + 2*J*Y(n)*replace_(n,n-1);


#procedure replace(n)
    argument if,1;
        mul replace_(n,`n');
    endargument;
    id if(1,a?,b?) = a;
    id if(0,a?,b?) = b;
    argument S;
        mul replace_(n,`n');
    endargument;
    id S(0) = 1;
    splitarg S;
    transform S,mulargs(1,last);
    mul replace_(n,`n');
    id S(a?) = a;
#endprocedure

argument Comm;
#call replace(15)*<---
endargument;
#call replace(15)*<---

splitarg Comm,1;
repeat;
id Comm(a?,a1?,?centr,b?) = Comm(a,b) + Comm(a1,?centr,b);
id Comm(a?,a1?,b?) = Comm(a,b)+Comm(a1,b);
endrepeat;
splitarg Comm,2;
repeat;
id Comm(a?,?centr,b1?,b?) = Comm(a,b) + Comm(a,?centr,b1);
id Comm(a?,b1?,b?) = Comm(a,b)+Comm(a,b1);
endrepeat;


#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim


Print;
.sort
''')

print(f.read('HIp1'))
print(f.read('HIm1'))
print(f.read('HX1'))
print(f.read('HY1'))
print(f.read('HM1'))


0
0
0
0
0


In [12]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
cfun f,S,sum;
Symbol i,k,a;

L L = S(sum(i,1,k,f(i)))*replace_(k,5);
argument S;
    id sum(?sss) = sum_(?sss);
endargument;
*mul replace_(k,5);
*argument;
*mul replace_(k,5);
*endargument;
*splitarg;
*transform S,mulargs(1,last);
*id S(k?) = k;
.sort
''')
f.read('L')    

'S(f(1)+f(2)+f(3)+f(4)+f(5))'

In [9]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
cfun f;
Symbol a,i,k;
.global
G G(k) = sum_(i,1,k,f(i));
.store

L L = G(a);
argument;
    id a=5;
endargument;
.sort
''')
f.read('L')    

'f(1)+f(2)+f(3)+f(4)+f(5)'

In [10]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
cfun f;
auto Symbols a,b;
auto Symbol n,i,j,k;

L L = sum_(i,1,k,f(i));
.sort
mul replace_(k,5);

.sort
''')
f.read('L')    

'f(1)+f(2)+f(3)+f(4)+f(5)'

In [2]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
f f;
s x,y,z,a;
l l = f(y,x,z,a);
rcyclesymmetrize f;
.sort
''')
f.read('l')

'f(x,y,a,z)'

In [97]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
auto s a,b,c;
CF Comm;
L L = Comm(a1+a2+a3+a4+a5,b1);

splitarg Comm,1;
repeat;
id Comm(a?,a1?,?centr,b?) = Comm(a,b) + Comm(a1,?centr,b);
id Comm(a?,a1?,b?) = Comm(a,b)+Comm(a1,b);
endrepeat;
splitarg Comm,2;
repeat;
id Comm(a?,?centr,b1?,b?) = Comm(a,b) + Comm(a,?centr,b1);
id Comm(a?,b1?,b?) = Comm(a,b)+Comm(a,b1);
endrepeat;

.sort
''')
f.read('L')

'Comm(a1,b1)+Comm(a2,b1)+Comm(a3,b1)+Comm(a4,b1)+Comm(a5,b1)'

In [140]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
auto s a,b,c;
CF Comm,S;
L L = Comm(-S(a1+a2+a3+a4+a5),b1);

id Comm(a?,b?) = a*Comm(b);

.sort
''')
f.read('L')

'-Comm(b1)*S(a5+a4+a3+a2+a1)'